### LABELED DATA

In [1]:
from utils.models import DataItem
lines =open('/home/ec2-user/SageMaker/mariano/notebooks/05. High Recall V2/labeled_data.csv','r').read().splitlines()[1:]
lines = [line.split(';') for line in lines ]
lines[:3]
labeled_data = [DataItem(id_) for id_,_ in lines]

idx = 0
for id_,label in lines:
        item = labeled_data[idx]
        idx+=1
        assert item.id_ == id_
        if label == 'R':
            item.set_relevant()
        else:
            item.set_irrelevant()
            assert label=='I'


lines = open('./auxiliary_notebooks/in_canada.csv', 'r').read().splitlines()
irrelevants = [DataItem(line.split(';')[0]) for line in lines if line.split(';')[1]=='False']
for item in irrelevants:
    item.set_irrelevant()
labeled_data = labeled_data + irrelevants
print(f'{len(labeled_data):,}')

114,492


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from utils.models import tokenize
from utils.tdmstudio import TDMStudio
import spacy
import string

def remove_punctuation(word):
    return ''.join([char for char in word if not char in string.punctuation+' '])

def tokenize(str_):
    tokens = [word.lemma_.lower() for word in nlp(str_) if not word.is_stop]
    tokens = [word.replace('\n', '') for word in tokens if not word.isnumeric() and len(remove_punctuation(word))!=0]
    return tokens


nlp = spacy.load('en_core_web_sm', disable=['textcat', 'parser','ner'])

stopwords = nlp.Defaults.stop_words
invalid = set([sw for sw in stopwords if any([token for token in tokenize(sw) if not token in stopwords ])]) # ['‘ve', "'m", '’ve', "'ve", '’m', '‘m', '‘d', '‘ll']
stopwords = set(stopwords.difference(invalid)) 
# vectorizer = TfidfVectorizer(
#                              input='content',
#                              lowercase=True,
#                              preprocessor=None,
#                              tokenizer=tokenize,
#                              analyzer='word',
#                              stop_words=list(stopwords),
# #                              token_pattern=r"(?u)\b\w\w+\b", #selects tokens of 2 or more alphanumeric char (punctuation is completely ignored and treated as token separator)
#                                                              # UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None
    
#                              ngram_range=(1,1), #lower and upper boundary of the range of n-values for different n-grams.
#                              max_df=1.0, #ignore terms that have a document frequency strictly higher than given threshold
#                              min_df=0.001, #ignore terms that have a document frequency strictly lower than given threshold
#                              max_features=10000, #build a vocabulary that only considers the top max_features ordered by term frequency acoss the corpus
#                              vocabulary=None, #vocabulary is determined from the input documents
#                              norm='l2',
#                              use_idf=False,
#                              )

In [8]:
from utils.tdmstudio import TDMStudio
corpus = map(TDMStudio.get_title_and_text, [item.filename() for item in labeled_data])
corpus = [f'{title}. {text}' for title, text in corpus]

In [7]:
from utils.general import info
info('Starting fit...')
X = vectorizer.fit_transform(corpus)
info('Getting vocab...')

2022-03-25 19:14:03.423843 [ INFO  ] Starting fit...
2022-03-25 19:57:18.705574 [ INFO  ] Getting vocab...


In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import pandas as pd
from utils.general import info
params = {'C':[0.5, 1, 5, 7, 15, 50]}
GS = GridSearchCV(
                  SVC(kernel='linear'),
                  params
                  ,scoring=['f1','accuracy','precision', 'recall'], 
                  cv=5,
                  verbose=4, 
                  refit='f1')    


# X = DataItem.get_X(labeled_data, type_=DataItem.TYPE_GLOVE600)
y = DataItem.get_y(labeled_data)
rta = GS.fit(X,y)
df = (pd.DataFrame(rta.cv_results_)[["params","mean_test_accuracy", "mean_test_precision","mean_test_recall","mean_test_f1"]])
df.columns=['params','mean accuracy', 'mean precision','mean recall','mean f1',]
print(df)
info(f'Best Params: {df.best_params_}')

clf_bow = SVC(kernel='linear', C=df.best_params_['C'], probability=True)
clf_bow.fit(X,y)
# clf_g600 = GS.best_estimator_

# del(X,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END C=0.5; accuracy: (test=0.804) f1: (test=0.050) precision: (test=0.025) recall: (test=1.000) total time=  20.8s
[CV 2/5] END C=0.5; accuracy: (test=0.999) f1: (test=0.840) precision: (test=1.000) recall: (test=0.724) total time=  50.4s
[CV 3/5] END C=0.5; accuracy: (test=0.999) f1: (test=0.868) precision: (test=1.000) recall: (test=0.767) total time=  48.9s
[CV 4/5] END C=0.5; accuracy: (test=0.999) f1: (test=0.851) precision: (test=1.000) recall: (test=0.741) total time=  49.0s
[CV 5/5] END C=0.5; accuracy: (test=0.999) f1: (test=0.879) precision: (test=1.000) recall: (test=0.784) total time=  49.5s
[CV 1/5] END C=1; accuracy: (test=0.794) f1: (test=0.047) precision: (test=0.024) recall: (test=1.000) total time=  18.7s
[CV 2/5] END C=1; accuracy: (test=0.999) f1: (test=0.863) precision: (test=1.000) recall: (test=0.759) total time=  46.5s
[CV 3/5] END C=1; accuracy: (test=0.999) f1: (test=0.907) precision: (test=0

AttributeError: 'DataFrame' object has no attribute 'best_params_'

In [4]:
from joblib import dump, load
# dump(vectorizer, 'vectorizer_bow_lastround.joblib') 
vectorizer = load('vectorizer_bow_lastround.joblib') 

In [9]:
# X = DataItem.get_X(labeled_data, type_=DataItem.TYPE_GLOVE600)
X = vectorizer.transform(corpus)
y = DataItem.get_y(labeled_data)



NameError: name 'SVC' is not defined

In [10]:
from sklearn.svm import SVC
clf = SVC(C=5, kernel='linear',probability=True)
clf.fit(X,y)

SVC(C=5, kernel='linear', probability=True)

In [7]:
list(vectorizer.vocabulary_)[:3]

['farm', 'job', 'want']

In [ ]:
# FALTA UN FIT con PROBA

In [11]:
import os
import pickle
import numpy as np

prediction_files = os.listdir('../04. Model of DP/predictions/')
suggestions = [DataItem(file_[:-5]) for file_ in  prediction_files]
print(suggestions[0])

DataItem(id=1323603426, source=GM1, label=U)


In [12]:

# prediction_files = os.listdir('../04. Model of DP/predictions/')
# suggestions = [DataItem(file_[:-5]) for file_ in  prediction_files]

# X = DataItem.get_X(suggestions, type_=DataItem.TYPE_GLOVE600)
unlabeled = map(TDMStudio.get_title_and_text, [item.filename() for item in suggestions])
unlabeled = [f'{title}. {text}' for title, text in unlabeled]
 
from utils.general import info
info('Starting fit...')
X = vectorizer.transform(unlabeled)
info('Getting vocab...')




2022-03-26 20:05:34.100508 [ INFO  ] Starting fit...
2022-03-26 20:07:22.016588 [ INFO  ] Getting vocab...


NameError: name 'clf_bow' is not defined

In [16]:
yhat = clf.predict_proba(X)

idx=0
for item in suggestions:
    prediction_file = '../04. Model of DP/predictions/'+item.id_+'_v3.p'
    assert os.path.isfile(prediction_file)
    
    y = pickle.load(open(prediction_file,'rb'))
    
    assert len(y)==3 and (y[2])==0 and y.shape==(3,)
#     print(y)
    y[2] = yhat[idx,1]
#     print(y)
#     print()
    pickle.dump(y, open(prediction_file,'wb'))
    idx+=1
#     if idx==5:
#         break

In [15]:
yhat[:5,:]

array([[0.45692272, 0.54307728],
       [0.5       , 0.5       ],
       [0.55830828, 0.44169172],
       [0.73855807, 0.26144193],
       [0.17539263, 0.82460737]])

In [ ]:
# prediction_files = os.listdir('../04. Model of DP/predictions/')
# suggestions = [DataItem(file_[:-5]) for file_ in  prediction_files]

# X = DataItem.get_X(suggestions, type_=DataItem.TYPE_GLOVE300)

# yhat = clf_g300.predict_proba(X)

# idx=0
# for item in suggestions:
    